In [1]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')
from decimal import *

In [2]:
# replacement={
#     '83/83':'1',    
# }

# # Replacement
# multi[curation_col]= multi[curation_col].apply(
#     lambda x: [replace_all(string, replacement) for string in x] if type(x)==list else x)

# matches=not_tube
# matches[curation_col]=matches['height'].astype(str)
# matches[curation_col]=matches[curation_col].apply(lambda x: f'"{x}"')
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'7\/165','.44',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))


# string = range_string
# lst = string.split('\n')
# builder_base = f'''
# def range_app(num):
# '''
# for val in lst:
#     if val.find(' - ') >-1:
#         original_val = val
#         lower, upper = val.split(' - ')
#         inner_f = f'''
#     if num >= {lower} and num <= {upper}:
#         return f'{lower}" - {upper}"{metric}'
#         '''
#         builder_base = builder_base + inner_f
#     if val.find('Greater than ') >-1:
#         original_val = val
#         upper = val.replace('Greater than ','')
#         inner_f = f'''
#     if num >= {upper}:
#         return f'{original_val}"{metric}'
#         '''
#         builder_base = builder_base + inner_f
# final_else = '''
#     else:
#         return r"n/a"
# '''
# exec(builder_base + final_else)
# print( builder_base + final_else)

# matches[curation_col]=matches['Min'].apply(range_app)
# matches['external_id']=matches['external_id'].apply('="{}"'.format)
# matches


# # Find a list of values to iterate
# df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
# df_strategy
# list_values=df_strategy['value'].to_list()
# # list_values.append('coarse')

# kwp=KeywordProcessor()
# kwp.add_keywords_from_list(list_values)
# # Extract Key Words
# df['matches']=df['long_desc'].apply(lambda x: kwp.extract_keywords(x)) #

In [3]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))  

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]  

def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

def natural_key(s, *args, **kwargs):
    return tuple(split_digits(s, *args, **kwargs))    

def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        print(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers

# str(round(num, 2))
def remove_fraction(numbers):
    try:
        if numbers.find('/') > -1:   
            inte, fract = numbers.split(' ')
            nom, den = fract.split('/')
            tmp = float(nom)/float(den)
            num = int(inte) + tmp
            return num
        else:
            return numbers
    except:
        return numbers

def remove_frac(numbers):
    if str(numbers).find('/') > -1:  
        nom, den = numbers.split('/')
        tmp = float(nom)/float(den)
        return tmp
    else:
        return numbers

def remove_exponent(num):
    return num.to_integral() if num == num.to_integral() else num.normalize()

In [4]:
customer_id = '5'
customer_name='%bedbathandbeyond%'
buckets = "Fixings & Fasteners"
# strategy_version_input=702
# attribute_id_input=2674
# formatted_attribute = 'weight'

In [5]:
# customer_id = '137'
# customer_name='%kimballwest%'
# buckets = "Fixings & Fasteners"
# strategy_version_input=705
# attribute_id_input=5130
# formatted_attribute = 'head_diameter_'

In [6]:
# parameters
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

NameError: name 'formatted_attribute' is not defined

In [ ]:
buckets = [x for x in enrich_db(f'''SELECT * FROM (SELECT * FROM strategy_buckets sb WHERE strategy_version_id = {strategy_version_input}) AS active_strat
INNER JOIN strategy_buckets_attributes sba ON active_strat.id = sba.strategy_bucket_id
WHERE sba.attribute_id = {attribute_id_input} 
AND sba.status = 'OPT_IN' ''')['bucket_id'].unique()] 

In [ ]:
df = query_from_file(file_name='./query/uncurated.sql', params=params)
print(len(df)) 
df = df[df['bucket_id'].isin(buckets)]
height=df
len(df)

In [ ]:
# Find a list of values to iterate
df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
df_strategy
list_values=df_strategy['value'].to_list()
# list_values.append('coarse')
print(list_values)
kwp=KeywordProcessor()
kwp.add_keywords_from_list(list_values)
# Extract Key Words
df['matches']=df['long_desc'].apply(lambda x: kwp.extract_keywords(x)) #
df['matches'].explode().value_counts()

In [ ]:
df['buckets'].explode().value_counts()[0:50] 

In [ ]:
regex_pattern_top=r'(?i)(top\s?head\s?dia.{0,15}\d+\s.{3})|(middle\s?head\s?dia.{0,15}\d+\s?.{3})|(bottom\s?head\s?dia.{0,15}\d+\s?.{3})|()'
regex_pattern_middle=r'(?i)(middle\s?head\s?dia.{0,20}\d+\s?.{3})|()'
regex_pattern_bottom=r'(?i)(bottom\s?head\s?dia.{0,20}\d+\s?.{3})|()'

df['matches_top']=df['long_desc'].apply(lambda x: re_extract(regex_pattern_top, x))
df['matches_middle']=df['long_desc'].apply(lambda x: re_extract(regex_pattern_middle, x))
df['matches_bottom']=df['long_desc'].apply(lambda x: re_extract(regex_pattern_bottom, x))
print(len(df[(df['matches_top'].astype(str)!='[]')])) 
print(len(df[(df['matches_middle'].astype(str)!='[]')])) 
print(len(df[(df['matches_bottom'].astype(str)!='[]')])) 
df_top=df[(df['matches_top'].astype(str)!='[]')]
df_top['new_matches_top']=df_top['matches_top']#.apply(lambda x: re.sub(r'[a-z]|[A-Z]|\:| ','',str(x)))#.apply(lambda x: re.sub(r"'\d+'",r'\d+ mm',str(x)))
df_top['new_matches_middle']=df_top['matches_middle'].apply(lambda x: re.sub(r'[a-z]|[A-Z]|\:| ','',str(x)))
df_top['new_matches_bottom']=df_top['matches_bottom'].apply(lambda x: re.sub(r'[a-z]|[A-Z]|\:| ','',str(x)))
df_top['matches_middle'].explode().value_counts()

In [ ]:
regex_pattern_top=r'(?i)(top\s?head\s?diam?e?t?e?r?.?\s?\d+\s?mm)|(middle\s?head\s?diam?e?t?e?r?.?\s?\s?\d+\s?mm)|(bottom\s?head\s?diam?e?t?e?r?.?\s?\d+\s?mm)|()'
regex_pattern_middle=r'(?i)(middle\s?head\s?dia.{0,20}\d+\s?mm)|()'
regex_pattern_bottom=r'(?i)(bottom\s?head\s?dia.{0,20}\d+\s?mm)|()'

df['matches_top']=df['long_desc'].apply(lambda x: re_extract(regex_pattern_top, x))
df['matches_middle']=df['long_desc'].apply(lambda x: re_extract(regex_pattern_middle, x))
df['matches_bottom']=df['long_desc'].apply(lambda x: re_extract(regex_pattern_bottom, x))
print(len(df[(df['matches_top'].astype(str)!='[]')])) 
print(len(df[(df['matches_middle'].astype(str)!='[]')])) 
print(len(df[(df['matches_bottom'].astype(str)!='[]')])) 
df_top=df[(df['matches_top'].astype(str)!='[]')]
df_top['new_matches_top']=df_top['matches_top'].apply(lambda x: re.sub(r'[a-z]|[A-Z]|\:| ','',str(x)))#.apply(lambda x: re.sub(r"'\d+'",r'\d+ mm',str(x)))
df_top['new_matches_middle']=df_top['matches_middle'].apply(lambda x: re.sub(r'[a-z]|[A-Z]|\:| ','',str(x)))
df_top['new_matches_bottom']=df_top['matches_bottom'].apply(lambda x: re.sub(r'[a-z]|[A-Z]|\:| ','',str(x)))
df_top['new_matches_top'].explode().value_counts()

In [ ]:
regex_pattern_head=r'(?i)(.{10}(?<=top )head\s?dia.{0,20}\d+\s?mm)|(.{10}(?<=bottom )head\s?dia.{0,20}\d+\s?mm)|(.{10}(?<=middle )head\s?dia.{0,20}\d+\s?mm)|()'
df['matches']=df['long_desc'].apply(lambda x: re_extract(regex_pattern_head, x))
print(len(df[(df['matches'].astype(str)!='[]')])) 
df_top=df[(df['matches'].astype(str)!='[]')]
df_top['new_matches']=df_top['matches'].apply(lambda x: re.sub(r'[a-z]|[A-Z]|\:| ','',str(x)))
df_top

In [ ]:
regex_pattern_head=r'(?i)(.{10}head\s?dia.{0,20}\d+\s?mm)|()'
df['matches']=df['long_desc'].apply(lambda x: re_extract(regex_pattern_head, x))
print(len(df[(df['matches'].astype(str)!='[]')])) 
df[(df['matches'].astype(str)!='[]')]

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',wipe)

# send to the folder for upload

In [ ]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
    matches.to_csv(f'{drive_path}/mult_buckets- {attribute}-- not_tube- upload_Shawn.csv',index=False)

In [ ]:
looks_good('Bed Bath & Beyond', attribute, df, not_tube)